In [78]:
import requests
import unicodedata

destinatario = '12946886000140'
api_brasil = requests.get(f'https://brasilapi.com.br/api/cnpj/v1/{destinatario}')
destino = api_brasil.json()
print(destino)

{'uf': 'PE', 'cep': '51170000', 'qsa': [{'pais': None, 'nome_socio': 'PAULO FERNANDO QUEIROZ DE FIGUEIREDO JUNIOR', 'codigo_pais': None, 'faixa_etaria': 'Entre 61 a 70 anos', 'cnpj_cpf_do_socio': '***123574**', 'qualificacao_socio': 'Presidente', 'codigo_faixa_etaria': 7, 'data_entrada_sociedade': '2014-02-04', 'identificador_de_socio': 2, 'cpf_representante_legal': '***000000**', 'nome_representante_legal': '', 'codigo_qualificacao_socio': 16, 'qualificacao_representante_legal': 'Não informada', 'codigo_qualificacao_representante_legal': 0}, {'pais': None, 'nome_socio': 'VANIA WALESKA TAVARES DE LIMA STUHRK', 'codigo_pais': None, 'faixa_etaria': 'Entre 61 a 70 anos', 'cnpj_cpf_do_socio': '***714384**', 'qualificacao_socio': 'Diretor', 'codigo_faixa_etaria': 7, 'data_entrada_sociedade': '2010-11-30', 'identificador_de_socio': 2, 'cpf_representante_legal': '***000000**', 'nome_representante_legal': '', 'codigo_qualificacao_socio': 10, 'qualificacao_representante_legal': 'Não informada',

In [79]:
def remover_acentos(texto):
    return ''.join(
        c for c in unicodedata.normalize('NFD', texto)
        if unicodedata.category(c) != 'Mn'
    )

def pegar_codigo_cidade(estado, cidade):
    
    # URL api Brasil para ibge
    url = f'https://brasilapi.com.br/api/ibge/municipios/v1/{estado}'
    requisicao = requests.get(url)
    dados_empresa = requisicao.json()
    for dicionario in dados_empresa:
        if cidade == remover_acentos(dicionario.get('nome')):
            cidade = dicionario.get('nome'), 
            codigo_cidade = dicionario.get('codigo_ibge')
    return cidade[0], codigo_cidade

In [80]:
destino.get('logradouro')

'MARECHAL MASCARENHAS DE MORAES'

In [89]:

itens_produtos = {
    "descricao": "Produto 1",
    "valor_unitario": 120
}

# Cálculo do valor total da NFe
quantidade = 5
valor_unitario = itens_produtos.get('valor_unitario')
valor_pagamento = float(valor_unitario * quantidade)


valor_unitario = float(valor_unitario)

In [90]:
valor_unitario

120.0

In [92]:
valor_pagamento

600.0

In [93]:
pegar_codigo_cidade(destino.get('uf'), destino.get('municipio'))

('RECIFE', '2611606')

In [94]:
# Como emitir 


cnpj = '15436220000130'
link = f'https://api.sandbox.plugnotas.com.br/nfe'

headers = {
    'x-api-key': '2da392a6-79d2-4304-a8b7-959572c7e44d'
}

dados_emissao = [{
    "emitente": {
        "cpfCnpj": cnpj,
        },
    
    "destinatario": {
        "cpfCnpj": destino.get('cnpj'),
        "endereco": {
            "bairro": destino.get('bairro'),
            "cep":destino.get('cep'),
            "codigoCidade" : pegar_codigo_cidade(destino.get('uf'), destino.get('municipio'))[1],
            "estado": destino.get('uf'),
            "logradouro": destino.get('logradouro'),
            "numero": destino.get('numero'),
            "tipoLogradouro": destino.get('descricao_tipo_de_logradouro'),            
        }
    },
    "itens": [{
        "descricao":  itens_produtos.get('descricao'),
        "ncm": "83100000",
        "cfop": "5102",
        "tributos": {
            "pis":{
                "cst": "99",
                "baseCalculo": {
                    "valor": 0.00
                },
                "aliquota": 0.00 
            },
            "cofins": {
                "cst": "99",
                "baseCalculo": {
                    "valor": 0.00
                },
                "aliquota": 0.00,
                "valor": 0.00,
                
            },
                    
        }, 
        
        "valorUnitario": {
             "comercial": float(valor_unitario),
             "tributavel": float(valor_unitario),
         }, 
    }],
   
    "pagamentos": {
        "aVista": True,
        "meio": "99",
        "valor": float(valor_pagamento),        
    },
},
]

emissao = requests.post(link, headers=headers, json=dados_emissao)

print(emissao.status_code)

print(emissao.text)

400
{"error":{"message":"Falha na validação do JSON de NFe","data":{"fields":{"NFe[undefined]":"Necessário enviar 'valorTroco' quando 'pagamentos.valor' for maior que o total da NFe"}}}}


In [95]:
float(valor_pagamento)

600.0

In [96]:
valor_unitario

120.0

In [97]:
# Solicitar Cancelamento da nota fiscal
idNota = '5f1f7069af3fbe5296dfxxx'
url_canc = f"https://api.sandbox.plugnotas.com.br/nfe/{idNota}/cancelamento"
justificativa_msg = "Erro na emissão da Nota Fiscal eletrônica.  2024"

justificativa = {
    "justificativa": justificativa_msg
}

cancelar = requests.post(url_canc, headers=headers, json=justificativa)
print(cancelar.status_code)
print(cancelar.text)

500
{"message":"Ocorreu um erro interno, nossa equipe já foi notificado do problema. Tente novamente mais tarde."}


In [98]:
# consultar status de nota cancelada

link_status = f"https://api.sandbox.plugnotas.com.br/nfe/{idNota}/cancelamento/status"

status = requests.get(link_status, headers=headers)
print(status.status_code)
print(status.text)

404
{"error":{"message":"Não localizamos qualquer NFe com os parâmetros informados","data":{"idOrChave":"5f1f7069af3fbe5296dfxxx"}}}


In [99]:
# Consultar resumo da nota
idNotaOrChaveOrProtocol = '26250115436220000130550010000040301146346848'
link_status = f"https://api.sandbox.plugnotas.com.br/nfe/{idNotaOrChaveOrProtocol}/resumo"

status = requests.get(link_status, headers=headers)
print(status.status_code)
print(status.text)

502
<html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
</body>
</html>



In [100]:
# Baixar PDF da nota
url_pdf = f'https://api.sandbox.plugnotas.com.br/nfe/{idNota}/pdf'
requisicao = requests.get(url_pdf, headers=headers)
print(requisicao.status_code)
print(requisicao.text)

404
{"error":{"message":"Não localizamos qualquer NFe com os parâmetros informados","data":{"id":"5f1f7069af3fbe5296dfxxx"}}}


In [101]:
idNotaOrChaveOrProtocol = '41201008187168000160558050010000131609769080'
idNotaOrChaveOrProtocol

'41201008187168000160558050010000131609769080'

In [102]:
# Baixar XML da nota
url_xml = f'https://api.sandbox.plugnotas.com.br/nfe/{idNotaOrChaveOrProtocol}/xml'
requisicao = requests.get(url_xml, headers=headers)
print(requisicao.status_code)
print(requisicao.text)

400
{"error":{"message":"Falha ao buscar NFe, o ID utilizado é inválido","data":{"id":"41201008187168000160558050010000131609769080"}}}


In [105]:
# Consultar notas por período
cnpj = '15436220000130'
url = f'https://api.sandbox.plugnotas.com.br/nfe/consulta/periodo?cpfCnpj={cnpj}&dataInicial=2024-12-01&dataFinal=2024-12-31'
nota_data = requisicao = requests.get(url, headers=headers)
print(nota_data.status_code)
print(nota_data.text)

200
{"hashProximaPagina":null,"notas":[]}


In [110]:
url_cidade = 'https://api.sandbox.plugnotas.com.br/nfse/cidades'
cidades = requisicao = requests.get(url_cidade, headers=headers)
print(cidades.status_code)


200


In [119]:
for cidade in cidades.json():
    if cidade.get('nome').lower() == 'recife':
        print(cidade)


{'id': 2611606, 'nome': 'Recife', 'uf': 'PE', 'padraoNacional': {'producao': False, 'homologacao': False}, 'padrao': 'TIPLAN', 'certificado': True, 'multiservicos': False, 'login': False, 'senha': False, 'lote': {}, 'sequencial': True, 'detalhesEmissao': {}, 'ambiente': {}, 'metodosDisponiveis': {}, 'token': False, 'detalhes': '', 'upload': False, 'descricao': 0, 'substituicao': True, 'informacoesComplementares': 0}
